In [37]:
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

In [38]:
#DATA TRIMESTRAL
#Colombia
col_querie = pd.read_excel(r'colombia_regiones_queries_.xlsx',header=9)
col_catego = pd.read_excel(r'colombia_regiones_categories_.xlsx',header=10)

#Ecuador
ecu_querie = pd.read_excel(r'ecuador_regiones_queries_.xlsx',header=9)
ecu_catego = pd.read_excel(r'ecuador_regiones_categories_.xlsx',header=10)

#Peru
per_querie = pd.read_excel(r'peru_regiones_queries_.xlsx',header=9)
per_catego = pd.read_excel(r'peru_regiones_categories_.xlsx',header=10)

#Chile
chi_querie = pd.read_excel(r'chile_regiones_queries_.xlsx',header=10)
chi_catego = pd.read_excel(r'chile_regiones_categories_.xlsx',header=11)


In [39]:
#FUNCIONES

def estructura_querie(df_querie, df_catego):

    def limpieza (df):
        # df.drop(columns='population',inplace=True)
        df = pd.melt(df,id_vars='weeks',
                    value_vars=df.columns[1:],
                    var_name='Fecha',
                    value_name='Total')
        
        df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date
        df['Fecha'] = [str(x) for x in df['Fecha']]
        df['ID'] = df['weeks'] + "_" + df['Fecha']
        df.sort_values(by='ID')

        return df 
# -------------------------------------------------------------------
    df_querie,df_catego = limpieza(df_querie),limpieza(df_catego)
    
    df = pd.merge(left      = df_querie,
                  right     = df_catego,
                  left_on   = 'ID',
                  right_on  = 'ID',
                  how = 'inner')
    
    df.rename(columns={'Total_x':'Total Menciones','Total_y':'Total Xenofobia','Fecha_y':'Fecha'},inplace=True)
    df[['Region','Pais']] = df['weeks_x'].str.split(',',expand=True)
# -------------------------------------------------------------------
    df['Tasa Xeno Regiones'] = df['Total Xenofobia'] / df['Total Menciones']
    df['Tasa Xeno Regiones'].fillna(0,inplace=True)
# -------------------------------------------------------------------

    df = df[['Pais','Region','Fecha','Total Menciones','Total Xenofobia','Tasa Xeno Regiones']]

    df.sort_values(by=['Pais'],inplace=True)
    df = df[(df['Region']!=(df['Pais'][0].strip())) & (df['Tasa Xeno Regiones'] != 0)]

    df.sort_values(by=['Region','Fecha'],inplace=True)

    return df

In [40]:
#INDICADOR PAIS
colombia = estructura_querie(col_querie,col_catego)
ecuador = estructura_querie(ecu_querie,ecu_catego)
peru = estructura_querie(per_querie,per_catego)
chile = estructura_querie(chi_querie,chi_catego)

In [ ]:
#EXPORTAR
# with pd.ExcelWriter('xenofobia_tasa_regiones_last_update.xlsx') as writer:
with pd.ExcelWriter('xenofobia_tasa_regiones_hub.xlsx') as writer:
    
    colombia.to_excel(writer,sheet_name='Colombia',index=False)
    ecuador.to_excel(writer,sheet_name='Ecuador',index=False)
    peru.to_excel(writer,sheet_name='Peru',index=False)
    chile.to_excel(writer,sheet_name='Chile',isndex=False)
